In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
# Load the dataset
df = pd.read_csv('/Users/zsl/Downloads/Customer-Churn.csv')

In [4]:
#
df.drop('customerID', axis=1, inplace=True)
# 
col_to_convert = ['Churn', 'Dependents', 'Partner','MultipleLines', 'PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']
for column in col_to_convert:
    df[column] = df[column].apply(lambda x: 1 if x == 'Yes' else (-1 if x == 'No internet service' or x == 'No phone service' else 0))

columns_to_encode = ['InternetService', 'Contract', 'PaymentMethod']
df_encoded = pd.get_dummies(df, columns=columns_to_encode)

#
df_encoded['gender'] = df_encoded['gender'].map({'Male': 0, 'Female': 1})

# Replace empty strings with NaN
df_encoded['TotalCharges'] = df_encoded['TotalCharges'].replace(' ', np.nan)
# Convert the 'TotalCharges' column to float
df_encoded['TotalCharges'] = df_encoded['TotalCharges'].astype(float)

In [5]:
df_encoded

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,1,0,-1,0,1,0,...,1,0,0,1,0,0,0,0,1,0
1,0,0,0,0,34,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,0,2,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,0,45,0,-1,1,0,1,...,1,0,0,0,1,0,1,0,0,0
4,1,0,0,0,2,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,0,1,1,24,1,1,1,0,1,...,1,0,0,0,1,0,0,0,0,1
7039,1,0,1,1,72,1,1,0,1,1,...,0,1,0,0,1,0,0,1,0,0
7040,1,0,1,1,11,0,-1,1,0,0,...,1,0,0,1,0,0,0,0,1,0
7041,0,1,1,0,4,1,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1


In [6]:
df = df_encoded

In [7]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'],
      dtype='object')

In [8]:
print(df.dtypes)

gender                                       int64
SeniorCitizen                                int64
Partner                                      int64
Dependents                                   int64
tenure                                       int64
PhoneService                                 int64
MultipleLines                                int64
OnlineSecurity                               int64
OnlineBackup                                 int64
DeviceProtection                             int64
TechSupport                                  int64
StreamingTV                                  int64
StreamingMovies                              int64
PaperlessBilling                             int64
MonthlyCharges                             float64
TotalCharges                               float64
Churn                                        int64
InternetService_DSL                          uint8
InternetService_Fiber optic                  uint8
InternetService_No             

In [9]:
df = df.dropna()

In [10]:
# Separate features and target variable
X = df.drop('Churn', axis=1)
y = df['Churn']

In [11]:
X = X.astype('float32')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

# Define the network architecture
def create_fnn_model(input_dim, output_dim, hidden_units):
    model = Sequential()
    model.add(Dense(hidden_units[0], activation='relu', input_dim=input_dim))
    
    # Add additional hidden layers if specified
    if len(hidden_units) > 1:
        for units in hidden_units[1:]:
            model.add(Dense(units, activation='relu'))
    
    model.add(Dense(output_dim, activation='softmax'))  # Softmax for multi-class classification
    
    return model

# Set the hyperparameters
input_dim = X.shape[1]  # Number of input features
output_dim = len(set(y))  # Number of output classes

# Define different network architectures
architectures = [[32], [50, 64], [27, 128, 32]]

# Define different activation functions
activation_functions = ['relu', 'sigmoid', 'tanh']

# Define different learning rates
learning_rates = [0.01, 0.001, 0.0001]

best_accuracy = 0.0
best_model = None
best_architecture = None
best_activation_function = None
best_learning_rate = None

# Train and evaluate models with different configurations
for architecture in architectures:
    for activation_function in activation_functions:
        for learning_rate in learning_rates:
            # Create the model
            model = create_fnn_model(input_dim, output_dim, architecture)
            
            # Compile the model with the chosen learning rate
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
            model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
            
            # Split the data into training and validation sets
            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
            
            # Train the model
            model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
            
            # Evaluate the model on the validation set
            _, accuracy = model.evaluate(X_val, y_val)
            
            # Track the best model based on validation accuracy
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model
                best_architecture = architecture
                best_activation_function = activation_function
                best_learning_rate = learning_rate
                
            # Print the results
            print(f"Architecture: {architecture} | Activation: {activation_function} | Learning Rate: {learning_rate}")
            print(f"Validation Accuracy: {accuracy}\n")

# Evaluate the best model on the test set
y_pred_prob = best_model.predict(X_test)
y_pred = y_pred_prob.argmax(axis=1)

# Calculate classification metrics
classification_metrics = classification_report(y_test, y_pred)
print("Classification Metrics:")
print(classification_metrics)

# Print the summary of the optimal model
print("Optimal Model Summary:")
best_model.summary()
print(f"\nArchitecture: {best_architecture} | Activation: {best_activation_function} | Learning Rate: {best_learning_rate}")

Epoch 1/10
176/176 [==============================] - 2s 5ms/step - loss: 10.5879 - accuracy: 0.7259 - val_loss: 20.8424 - val_accuracy: 0.7704
Epoch 2/10
176/176 [==============================] - 1s 4ms/step - loss: 6.8815 - accuracy: 0.7358 - val_loss: 2.6957 - val_accuracy: 0.7825
Epoch 3/10
176/176 [==============================] - 1s 3ms/step - loss: 5.2754 - accuracy: 0.7356 - val_loss: 3.1457 - val_accuracy: 0.7846
Epoch 4/10
176/176 [==============================] - 1s 4ms/step - loss: 5.0700 - accuracy: 0.7413 - val_loss: 1.3401 - val_accuracy: 0.7932
Epoch 5/10
176/176 [==============================] - 1s 3ms/step - loss: 5.5823 - accuracy: 0.7300 - val_loss: 1.5928 - val_accuracy: 0.6631
Epoch 6/10
176/176 [==============================] - 0s 3ms/step - loss: 1.7725 - accuracy: 0.7570 - val_loss: 9.6093 - val_accuracy: 0.4534
Epoch 7/10
176/176 [==============================] - 1s 3ms/step - loss: 1.6309 - accuracy: 0.7552 - val_loss: 1.5172 - val_accuracy: 0.7811
Epoc